# ETL DE LOS DATASETS MOVIES Y CREDITS

In [1]:
# Librerias necesarias

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ast

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Cargar los datasets

df_movies = pd.read_csv('movies_dataset.csv', encoding = "utf-8")
df_credits = pd.read_csv('credits.csv', encoding ='utf-8')

## DATASETS MOVIES

Se observa la estructura, información y contenido del dataframe.

In [4]:
pd.set_option('display.max_columns', None)
df_movies.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [5]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [6]:
# Verificar qué valores de "id" no son numéricos
non_numeric_ids = df_movies[~df_movies['id'].str.isdigit()]['id']
print(non_numeric_ids)


19730    1997-08-20
29503    2012-09-29
35587    2014-01-01
Name: id, dtype: object


In [7]:
# Eliminar las filas donde 'id' no es numérico
df_movies = df_movies[df_movies['id'].str.isdigit()]


In [8]:
# Tranformo el tipo de datos de "id" a int para poder utilizarlo como clave de unión posteriormente.
df_movies['id'] = df_movies['id'].astype("int64")

## DESANIDADO DE DATOS

Se observa que las columnas "belongs_to_collection", "genres", "production_companies", "production_countries" y "spoken_languages" contienen datos anidados. Se desanidan todos los datos y se vinculan solo los necesarios con la informacion original en un nuevo dataframe llamado "df_expand".

### Columna "belongs_to_collection"

In [9]:
import ast

# Desanidar "belongs_to_collection" en un nuevo dataframe llamado "df_collection"

# Filtrar filas no nulas
df_collection = df_movies[['id', 'belongs_to_collection']].dropna()

# Convertir string a diccionario
df_collection['belongs_to_collection'] = df_collection['belongs_to_collection'].apply(ast.literal_eval)

# Expandir la columna
df_collection_expanded = pd.json_normalize(df_collection['belongs_to_collection'])

# Renombrar las columnas expandidas para evitar solapamientos
df_collection_expanded = df_collection_expanded.add_prefix('collection_')

# Concatenar con el DataFrame original de colección
df_collection = pd.concat([df_collection[['id']], df_collection_expanded], axis=1)

# Verificar el resultado
df_collection.head()


,id,collection_id,collection_name,collection_poster_path,collection_backdrop_path
0,862.0,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg
2,15602.0,96871.0,Father of the Bride Collection,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg
4,11862.0,117693.0,Balto Collection,/w0ZgH6Lgxt2bQYnf1ss74UvYftm.jpg,/9VM5LiJV0bGb1st1KyHA3cVnO2G.jpg
9,710.0,136214.0,Pocahontas Collection,/s4RkDe4AjwyrlltdF6Ui7INlME6.jpg,/qqeOxNmtAW7WXdbrmQftgluvve7.jpg
12,21032.0,10924.0,From Dusk Till Dawn Collection,/a156nAy5ZTGXlO5wgfvnF26aaDB.jpg,/oIJQPVy33uxBAu9LMXwjsgHLyxP.jpg


In [10]:
# Se eliminan las columnas innecesarias, dejando solamente el "id" para realizar el merge y "collection_name"

df_collection = df_collection.drop(columns=['collection_id', 'collection_poster_path', 'collection_backdrop_path'])


In [11]:
df_collection.head(2)

,id,collection_name
0,862.0,Toy Story Collection
2,15602.0,Father of the Bride Collection


In [12]:
# Se genera un df nuevo llamado "df_expand" donde se agregan las columnas desanidadas de la columna "belongs_to_collection", utilizando la columna "id" como clave de union entre ambos dataframes.

# Unir los DataFrames usando merge
df_expand = pd.merge(df_movies, df_collection, on='id', how='left')

# Verificar el resultado
df_expand.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,collection_name
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN


In [13]:
# Eliminar la columna 'belongs_to_collection'
df_expand = df_expand.drop('belongs_to_collection', axis=1)

### Columna "genres"

La columna "genres" en algunos casos presenta listas vacias, en otras listas con un diccionario y en otras listas con más de un diccionario en su interior. Directamente se modifica la columna 'genres' de manera que ahora contiene listas de nombres de géneros en lugar de listas de diccionarios completos.

In [15]:
# Se convierten las cadenas de texto que representan listas (o diccionarios) en estructuras de lista de Python utilizando ast.literal_eval.
df_movies['genres'] = df_movies['genres'].apply(ast.literal_eval)

# Se extraen los nombres de los géneros de la lista de diccionarios resultante, almacenando solo los nombres en lugar de los diccionarios completos en mi dataframe "df_expand".
df_expand['genres'] = df_movies['genres'].apply(lambda x: [i['name'] for i in x])


In [16]:
df_expand.head(10)

,adult,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,collection_name
0,False,30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection
1,False,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN
2,False,0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,Father of the Bride Collection
3,False,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,NaN
4,False,0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,Balto Collection
5,False,60000000,"[Action, Crime, Drama, Thriller]",NaN,949,tt0113277,en,Heat,"Obsessive master thief, Neil McCauley leads a ...",17.924927,/zMyfPUelumio3tiDKPffaUpsQTD.jpg,"[{'name': 'Regency Enterprises', 'id': 508}, {...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,187436818.0,170.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,A Los Angeles Crime Saga,Heat,False,7.7,1886.0,NaN
6,False,58000000,"[Comedy, Romance]",NaN,11860,tt0114319,en,Sabrina,An ugly duckling having undergone a remarkable...,6.677277,/jQh15y5YB7bWz1NtffNZmRw0s9D.jpg,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'DE', 'name': 'Germany'}, {'is...",1995-12-15,0.0,127.0,"[{'iso_639_1': 'fr', 'name': 'Français'}, {'is...",Released,You are cordially invited to the most surprisi...,Sabrina,False,6.2,141.0,NaN
7,False,0,"[Action, Adventure, Drama, Family]",NaN,45325,tt0112302,en,Tom and Huck,"A mischievous young boy, Tom Sawyer, witnesses...",2.561161,/sGO5Qa55p7wTu7FJcX4H4xIVKvS.jpg,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,97.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,The Original Bad Boys.,Tom and Huck,False,5.4,45.0,NaN
8,False,35000000,"[Action, Adventure, Thriller]",NaN,9091,tt0114576,en,Sudden Death,International action superstar Jean Claude Van...,5.23158,/eoWvKD60lT95Ss1MYNgVExpo5iU.jpg,"[{'name': 'Universal Pictures', 'id': 33}, {'n...","[{'iso_3166_1': 'US', 'name

### Columna "spoken_language"

La columna "spoken_language" en algunos casos presenta listas vacias, en otras listas con un diccionario y en otras listas con más de un diccionario en su interior. Sólo se conserva el nombre del lenguaje, o sea el "name" ubicado dentro del diccionario.

In [18]:
df_movies['spoken_languages'].apply(type).value_counts()

spoken_languages
<class 'str'>      45460
<class 'float'>        3
Name: count, dtype: int64

In [19]:
df_movies['spoken_languages'].isnull().sum()


3

In [20]:
#Elimino nulos y compruebo nuevamente que desaparecieron los float.

df_movies = df_movies.dropna(subset = ['spoken_languages'])

In [21]:
df_movies['spoken_languages'].apply(type).value_counts()

spoken_languages
<class 'str'>    45460
Name: count, dtype: int64

In [22]:
# Se convierten los datos en estructuras de listas de python.
df_movies['spoken_languages'] = df_movies['spoken_languages'].apply(ast.literal_eval)

# Crear una lista para almacenar los nuevos registros de lenguajes
language_data = []

# Iterar sobre las filas del dataframe original
for idx, row in df_movies.iterrows():
    movie_id = row['id']
    languages = row['spoken_languages']
    if languages:  # Si hay idiomas listados
        for lang in languages:
            language_data.append({'id': movie_id, 'language_iso': lang['iso_639_1'], 'language_name': lang['name']})

# Crear el nuevo dataframe df_languages
df_languages = pd.DataFrame(language_data)

# Mostrar las primeras filas del nuevo dataframe
df_languages.head()


,id,language_iso,language_name
0,862,en,English
1,8844,en,English
2,8844,fr,Français
3,15602,en,English
4,31357,en,English


Me quedo solo con la columna "id" como clave de unión y con "language_name" para agregar al dataframe "df_expand"

In [23]:
# Eliminar la columna 'language_iso'
df_languages.drop(columns=['language_iso'], inplace=True)

In [24]:
df_languages

,id,language_name
0,862,English
1,8844,English
2,8844,Français
3,15602,English
4,31357,English
...,...,...
53295,30840,English
53296,439050,فارسی
53297,111109,
53298,67758,English


In [25]:
# Unir los datos desanidados en "df_languages" con "df_expand"

# Realizar el merge
df_expand = pd.merge(df_expand, df_languages, on='id', how='left')

# Mostrar las primeras filas del nuevo dataframe resultante
df_expand.head(2)

,adult,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,collection_name,language_name
0,False,30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection,English
1,False,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN,English


In [26]:
# Eliminar la columna 'spoken_languages'
df_expand = df_expand.drop('spoken_languages', axis=1)

In [27]:
# Cambiar el nombre de la columna "language_name" a "spoken_languages"
df_expand.rename(columns={'language_name': 'spoken_languages'}, inplace=True)

In [28]:
df_expand.head(2)

,adult,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,status,tagline,title,video,vote_average,vote_count,collection_name,spoken_languages
0,False,30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection,English
1,False,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN,English


### Columna "production_companies"

In [29]:
# Ver si hay nulos
df_movies['production_companies'].isnull().sum()


0

In [30]:
#Convierto el string contenido en 'production_companies' a listas de Python
df_movies['production_companies'] = df_movies['production_companies'].apply(ast.literal_eval)

In [31]:
# Corroboro que sean clase list
df_movies['production_companies'].apply(type).value_counts()

production_companies
<class 'list'>    45460
Name: count, dtype: int64

In [32]:
df_movies['production_companies'].head(10)

0       [{'name': 'Pixar Animation Studios', 'id': 3}]
1    [{'name': 'TriStar Pictures', 'id': 559}, {'na...
2    [{'name': 'Warner Bros.', 'id': 6194}, {'name'...
3    [{'name': 'Twentieth Century Fox Film Corporat...
4    [{'name': 'Sandollar Productions', 'id': 5842}...
5    [{'name': 'Regency Enterprises', 'id': 508}, {...
6    [{'name': 'Paramount Pictures', 'id': 4}, {'na...
7          [{'name': 'Walt Disney Pictures', 'id': 2}]
8    [{'name': 'Universal Pictures', 'id': 33}, {'n...
9    [{'name': 'United Artists', 'id': 60}, {'name'...
Name: production_companies, dtype: object

In [33]:
# Esta función me permitirá extraer los valores "name" dentro de los diccionarios contenidos en las listas.
def extract_names(x):
    try:
        companies_names = ast.literal_eval(x) if isinstance(x, str) else x
        return [i['name'] for i in companies_names if isinstance(i, dict) and 'name' in i]
    except:
        return []

# Se aplica la función a la columna
df_expand['production_companies'] = df_movies['production_companies'].apply(extract_names)

# Verificar los resultados
df_expand['production_companies'].head(10)


0                            [Pixar Animation Studios]
1    [TriStar Pictures, Teitler Film, Interscope Co...
2                       [Warner Bros., Lancaster Gate]
3             [Twentieth Century Fox Film Corporation]
4         [Sandollar Productions, Touchstone Pictures]
5    [Regency Enterprises, Forward Pass, Warner Bros.]
6    [Paramount Pictures, Scott Rudin Productions, ...
7                               [Walt Disney Pictures]
8    [Universal Pictures, Imperial Entertainment, S...
9                    [United Artists, Eon Productions]
Name: production_companies, dtype: object

In [34]:
df_expand.head(2)

,adult,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,status,tagline,title,video,vote_average,vote_count,collection_name,spoken_languages
0,False,30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,[Pixar Animation Studios],"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection,English
1,False,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[TriStar Pictures, Teitler Film, Interscope Co...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN,English


### Columna "production_countries"

In [35]:
# Ver tipos de datos y cantidad de valores
df_movies['production_countries'].apply(type).value_counts()

production_countries
<class 'str'>    45460
Name: count, dtype: int64

In [36]:
# Ver si hay nulos
df_movies['production_countries'].isnull().sum()

0

In [37]:
#Definir una funcion para manejar los errores al momento de convertir los datos de string a lista y luego extraer los "name" de paises. 

def string_tolist(x):
    if isinstance(x, str):
        try:
            data = ast.literal_eval(x)  
            return [i['name'] for i in data if isinstance(i, dict) and 'name' in i]
        except:
            return []
    else:
        return []
    

#Aplicar la funcion a la columna
df_expand['production_countries'] = df_movies['production_countries'].apply(string_tolist)


In [38]:
#Corroborar la conversion
df_expand['production_countries'].head(10)

0                    [United States of America]
1                    [United States of America]
2                    [United States of America]
3                    [United States of America]
4                    [United States of America]
5                    [United States of America]
6           [Germany, United States of America]
7                    [United States of America]
8                    [United States of America]
9    [United Kingdom, United States of America]
Name: production_countries, dtype: object

In [39]:
# Ver si hay duplicados en "id"
duplicates = df_expand[df_expand.duplicated(['id'], keep=False)]

if not duplicates.empty:
    # Si hay duplicados, se eliminan manteniendo solo la primera ocurrencia
    df_expand.drop_duplicates(subset=['id'], keep='first', inplace=True)
    print("Se eliminaron los duplicados en la columna 'id'.")
else:
    print("No se encontraron duplicados en la columna 'id'.")



Se eliminaron los duplicados en la columna 'id'.


### TRANSFORMACIONES REQUERIDAS

Luego de desanidar y agregar al df_expand todos los datos necesarios se procede con el resto de las transformaciones requeridas.

In [40]:
df_expand.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45433 entries, 0 to 57213
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   adult                 45433 non-null  object 
 1   budget                45433 non-null  object 
 2   genres                45427 non-null  object 
 3   homepage              7774 non-null   object 
 4   id                    45433 non-null  int64  
 5   imdb_id               45416 non-null  object 
 6   original_language     45422 non-null  object 
 7   original_title        45433 non-null  object 
 8   overview              44479 non-null  object 
 9   popularity            45430 non-null  object 
 10  poster_path           45047 non-null  object 
 11  production_companies  35327 non-null  object 
 12  production_countries  35327 non-null  object 
 13  release_date          45346 non-null  object 
 14  revenue               45430 non-null  float64
 15  runtime               45

Rellenar los valores nulos de los campos revenue, budget por el número 0.

In [41]:
# Ver el tipo de dato específico de la columna "budget"
print(df_expand['budget'].dtype)


object


In [42]:
# Convertir la columna "budget" a tipo numérico
df_expand['budget'] = pd.to_numeric(df_expand['budget'], errors='coerce')

In [43]:
# Verificar de nuevo los tipos de datos después de la conversión
print(df_expand['budget'].dtype)

int64


In [44]:
# Rellenar los valores nulos en revenue y budget con 0
df_expand['revenue'].fillna(0, inplace=True)
df_expand['budget'].fillna(0, inplace=True)

De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

In [45]:
# Convertir la columna 'release_date' a tipo datetime
df_expand['release_date'] = pd.to_datetime(df_expand['release_date'], errors='coerce')

Se eliminan los valores nulos del campo "release_date".

In [46]:
df_expand['release_date'].isnull().sum()

87

In [47]:
df_expand = df_expand.dropna(subset=['release_date'])

In [48]:
df_expand['release_date'].isnull().sum()

0

In [49]:
# Crear la columna "release_year" con el año de lanzamiento de las peliculas.
df_expand['release_year'] = df_expand['release_date'].dt.year

Se crea la columna con el retorno de inversión, llamada "return" con los campos "revenue" y "budget", dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

In [50]:
# Calcular la columna 'return' con la fórmula y manejo de NaN. O sea, si el calculo no se puede realizar se coloca 0.
df_expand['return'] = df_expand.apply(lambda row: row['revenue'] / row['budget'] if row['budget'] != 0 else 0, axis=1)

In [51]:
df_expand.head(2)

,adult,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,status,tagline,title,video,vote_average,vote_count,collection_name,spoken_languages,release_year,return
0,False,30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,[Pixar Animation Studios],[United States of America],1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection,English,1995,12.451801
1,False,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN,English,1995,4.043035


Eliminar las columnas que no serán utilizadas, video, imdb_id, adult, original_title, poster_path y homepage.

In [52]:
df_expand.drop(['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage'], axis=1, inplace=True)


## DATASET CREDITS

In [53]:
df_credits.head(2)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844


In [54]:
df_credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [55]:
# Convierto las columnas "cast" y "crew" a listas de Python

df_credits['cast'] = df_credits['cast'].apply(ast.literal_eval)
df_credits['crew'] = df_credits['crew'].apply(ast.literal_eval) 

In [56]:
# Función para obtener la lista de nombres de actores

def actor_names(cast_list):
    return [actor['name'] for actor in cast_list]

# Aplicar la función a la columna 'cast' y crear la columna 'actors'
df_credits['actors'] = df_credits['cast'].apply(lambda x: actor_names(x))

# Mostrar el DataFrame resultante
df_credits.head()

,cast,crew,id,actors
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney..."
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,..."
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop..."
3,"[{'cast_id': 1, 'character': 'Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,"[Whitney Houston, Angela Bassett, Loretta Devi..."
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,"[Steve Martin, Diane Keaton, Martin Short, Kim..."


In [57]:
# Función para obtener los nombres de directores

def director_name(crew_list):
    for director in crew_list:
        if director['job'] == 'Director':
            return director['name']
    return None

#Aplico la funcion y creo una nueva columna 'director'
df_credits['director'] = df_credits['crew'].apply(director_name)

In [58]:
# Elimino las columnas innecesarias
df_credits.drop(['cast', 'crew'], axis=1, inplace=True)

In [59]:
df_credits.head()

,id,actors,director
0,862,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",John Lasseter
1,8844,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",Joe Johnston
2,15602,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...",Howard Deutch
3,31357,"[Whitney Houston, Angela Bassett, Loretta Devi...",Forest Whitaker
4,11862,"[Steve Martin, Diane Keaton, Martin Short, Kim...",Charles Shyer


In [60]:
# Agregar columnas de director y actores a df_expand
df_expand = pd.merge(df_expand, df_credits, on='id', how='left')

# Mostrar las primeras filas del nuevo dataframe resultante
df_expand.head(2)

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,collection_name,spoken_languages,release_year,return,actors,director
0,30000000,"[Animation, Comedy, Family]",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,[Pixar Animation Studios],[United States of America],1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,7.7,5415.0,Toy Story Collection,English,1995,12.451801,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",John Lasseter
1,65000000,"[Adventure, Fantasy, Family]",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,NaN,English,1995,4.043035,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",Joe Johnston


In [61]:
df_expand.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45390 entries, 0 to 45389
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   budget                45390 non-null  int64         
 1   genres                45385 non-null  object        
 2   id                    45390 non-null  int64         
 3   original_language     45379 non-null  object        
 4   overview              44449 non-null  object        
 5   popularity            45390 non-null  object        
 6   production_companies  35328 non-null  object        
 7   production_countries  35328 non-null  object        
 8   release_date          45390 non-null  datetime64[ns]
 9   revenue               45390 non-null  float64       
 10  runtime               45144 non-null  float64       
 11  status                45310 non-null  object        
 12  tagline               20401 non-null  object        
 13  title           

In [62]:
# Mostrar los tipos de datos de cada columna
print(df_expand.dtypes)


budget                           int64
genres                          object
id                               int64
original_language               object
overview                        object
popularity                      object
production_companies            object
production_countries            object
release_date            datetime64[ns]
revenue                        float64
runtime                        float64
status                          object
tagline                         object
title                           object
vote_average                   float64
vote_count                     float64
collection_name                 object
spoken_languages                object
release_year                     int32
return                         float64
actors                          object
director                        object
dtype: object


In [63]:
# Resumen estadístico de columnas numéricas
df_expand.describe()

,budget,id,release_date,revenue,runtime,vote_average,vote_count,release_year,return
count,4.539000e+04,45390.000000,45390,4.539000e+04,45144.000000,45390.000000,45390.000000,45390.000000,4.539000e+04
mean,4.231336e+06,108019.723838,1992-05-17 03:37:58.968935936,1.122664e+07,94.171474,5.624166,110.069487,1991.885151,6.598392e+02
min,0.000000e+00,2.000000,1874-12-09 00:00:00,0.000000e+00,0.000000,0.000000,0.000000,1874.000000,0.000000e+00
25%,0.000000e+00,26380.250000,1978-10-08 00:00:00,0.000000e+00,85.000000,5.000000,3.000000,1978.000000,0.000000e+00
50%,0.000000e+00,59857.500000,2001-08-31 12:00:00,0.000000e+00,95.000000,6.000000,10.000000,2001.000000,0.000000e+00
75%,0.000000e+00,156500.500000,2010-12-17 00:00:00,0.000000e+00,107.000000,6.800000,34.000000,2010.000000,0.000000e+00
max,3.800000e+08,469172.000000,2020-12-16 00:00:00,2.787965e+09,1256.000000,10.000000,14075.000000,2020.000000,1.239638e+07
std,1.743732e+07,112158.635262,NaN,6.438033e+07,38.347785,1.915447,491.669661,24.053411,7.468177e+04


In [64]:
# Conteo de valores no nulos por columna
print(df_expand.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45390 entries, 0 to 45389
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   budget                45390 non-null  int64         
 1   genres                45385 non-null  object        
 2   id                    45390 non-null  int64         
 3   original_language     45379 non-null  object        
 4   overview              44449 non-null  object        
 5   popularity            45390 non-null  object        
 6   production_companies  35328 non-null  object        
 7   production_countries  35328 non-null  object        
 8   release_date          45390 non-null  datetime64[ns]
 9   revenue               45390 non-null  float64       
 10  runtime               45144 non-null  float64       
 11  status                45310 non-null  object        
 12  tagline               20401 non-null  object        
 13  title           

In [65]:
# Contar valores nulos por columna
print(df_expand.isnull().sum())


budget                      0
genres                      5
id                          0
original_language          11
overview                  941
popularity                  0
production_companies    10062
production_countries    10062
release_date                0
revenue                     0
runtime                   246
status                     80
tagline                 24989
title                       0
vote_average                0
vote_count                  0
collection_name         44621
spoken_languages         3771
release_year                0
return                      0
actors                      1
director                  836
dtype: int64


In [66]:
# Contar valores duplicados en la columna "id"
print(df_expand['id'].duplicated().sum())

# Eliminar duplicados dejando la primera aparición
df_expand = df_expand.drop_duplicates(subset='id', keep='first')

44


In [67]:
# Eliminar registros con datos nulos en las columnas "actors" y "director"
df_expand = df_expand.dropna(subset=["actors", "director"])

In [68]:
df_expand.head(2)

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,collection_name,spoken_languages,release_year,return,actors,director
0,30000000,"[Animation, Comedy, Family]",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,[Pixar Animation Studios],[United States of America],1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,7.7,5415.0,Toy Story Collection,English,1995,12.451801,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",John Lasseter
1,65000000,"[Adventure, Fantasy, Family]",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,NaN,English,1995,4.043035,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",Joe Johnston


Para reducir la cantidad de datos se genera un "df_filtrado" que solo contendrá las peliculas con idioma original en ingles y en español.

In [69]:
# Valores únicos en la columna 'original_language'
valores_originales = df_expand['original_language'].unique()

# Valores únicos en la columna 'spoken_language'
valores_hablados = df_expand['spoken_languages'].unique()

# Convertir a listas si lo necesitas
valores_originales_lista = list(valores_originales)
valores_hablados_lista = list(valores_hablados)

# Mostrar los valores únicos
print("Valores únicos en 'original_language':")
print(valores_originales_lista)

print("\nValores únicos en 'spoken_languages':")
print(valores_hablados_lista)


Valores únicos en 'original_language':
['en', 'fr', 'zh', 'it', 'fa', 'nl', 'de', 'cn', 'ar', 'es', 'ru', 'sv', 'ja', 'ko', 'sr', 'bn', 'he', 'pt', 'wo', 'ro', 'hu', 'cy', 'vi', 'cs', 'da', 'no', 'nb', 'pl', 'el', 'sh', 'xx', 'mk', 'bo', 'ca', 'fi', 'th', 'sk', 'bs', 'hi', 'tr', 'is', 'ps', 'ab', 'eo', 'ka', 'mn', 'bm', 'zu', 'uk', 'af', 'la', 'et', 'ku', 'fy', 'lv', 'ta', 'sl', 'tl', 'ur', 'rw', 'id', 'bg', 'mr', 'lt', 'kk', 'ms', 'sq', nan, 'qu', 'te', 'am', 'jv', 'tg', 'ml', 'hr', 'lo', 'ay', 'kn', 'eu', 'ne', 'pa', 'ky', 'gl', 'uz', 'sm', 'mt', 'hy', 'iu', 'lb', 'si']

Valores únicos en 'spoken_languages':
['English', 'Français', '广州话 / 廣州話', '普通话', 'Pусский', 'Español', nan, 'shqip', 'Italiano', 'Deutsch', 'فارسی', 'Nederlands', 'Dansk', 'العربية', 'Magyar', '', 'Český', 'svenska', '日本語', 'Português', 'Català', '한국어/조선말', 'Afrikaans', 'বাংলা', 'עִבְרִית', 'Latin', 'Cymraeg', 'Tiếng Việt', 'Polski', 'български език', 'ελληνικά', 'Norsk', 'Bosanski', 'Gaeilge', 'Bokmål', 'Українськи

In [70]:

# Creas una lista con los idiomas que deseas filtrar
idiomas_deseados = ['en', 'es']

# Filtras el DataFrame usando el método isin()
df_filtrado = df_expand[df_expand['original_language'].isin(idiomas_deseados)]

# Ahora df_filtrado contiene solo los registros donde original_language está en la lista idiomas_deseados
df_filtrado


,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,collection_name,spoken_languages,release_year,return,actors,director
0,30000000,"[Animation, Comedy, Family]",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,[Pixar Animation Studios],[United States of America],1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,7.7,5415.0,Toy Story Collection,English,1995,12.451801,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",John Lasseter
1,65000000,"[Adventure, Fantasy, Family]",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,NaN,English,1995,4.043035,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",Joe Johnston
2,0,"[Romance, Comedy]",15602,en,A family wedding reignites the ancient feud be...,11.7129,[Twentieth Century Fox Film Corporation],[United States of America],1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,Father of the Bride Collection,English,1995,0.000000,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...",Howard Deutch
3,16000000,"[Comedy, Drama, Romance]",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,"[Sandollar Productions, Touchstone Pictures]",[United States of America],1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,NaN,English,1995,5.090760,"[Whitney Houston, Angela Bassett, Loretta Devi...",Forest Whitaker
4,0,[Comedy],11862,en,Just when George Banks has recovered from his ...,8.387519,"[Regency Enterprises, Forward Pass, Warner Bros.]",[United States of America],1995-02-10,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,Balto Collection,English,1995,0.000000,"[Steve Martin, Diane Keaton, Martin Short, Kim...",Charles Shyer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45384,0,[Drama],222848,en,It's the year 3000 AD. The world's most danger...,0.661558,NaN,NaN,1995-01-01,0.0,85.0,Released,NaN,Caged Heat 3000,3.5,1.0,NaN,English,1995,0.000000,"[Lisa Boyle, Kena Land, Zaneta Polard, Don Yan...",Aaron Osborne
45385,0,"[Action, Drama, Thriller]",30840,en,"Yet another version of the classic epic, with ...",5.683753,NaN,NaN,1991-05-13,0.0,104.0,Released,NaN,Robin Hood,5.7,26.0,NaN,English,1991,0.000000,"[Patrick Bergin, Uma Thurman, David Morrissey,...",John Irvin
45387,0,NaN,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,NaN,NaN,2003-08-01,0.0,90.0,Released,A deadly game of wits.,Betrayal,3.8,6.0,NaN,English,2003,0.000000,"[Erika Eleniak, Adam Baldwin, Julie du Page, J...",Mark L. Lester
45388,0,NaN,227506,en,"In a small town live two brothers, one a minis...",0.003503,NaN,NaN,1917-10-21,0.0,87.0,Released,NaN,Satan Triumphant,0.0,0.0,NaN,NaN,1917,0.000000,"[Iwan Mosschuchin, Nathalie Lissenko, Pavel Pa...",Yakov Protazanov


In [71]:
df_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32484 entries, 0 to 45389
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   budget                32484 non-null  int64         
 1   genres                32480 non-null  object        
 2   id                    32484 non-null  int64         
 3   original_language     32484 non-null  object        
 4   overview              32358 non-null  object        
 5   popularity            32484 non-null  object        
 6   production_companies  26301 non-null  object        
 7   production_countries  26301 non-null  object        
 8   release_date          32484 non-null  datetime64[ns]
 9   revenue               32484 non-null  float64       
 10  runtime               32456 non-null  float64       
 11  status                32433 non-null  object        
 12  tagline               17996 non-null  object        
 13  title                

In [72]:
unique_popularity_types = df_expand['popularity'].apply(type).unique()
print(unique_popularity_types)


[<class 'float'> <class 'str'>]


In [73]:
# Identificar y corregir valores 'str' en 'popularity' convirtiéndolos a 'float'
df_expand['popularity'] = pd.to_numeric(df_expand['popularity'], errors='coerce')



In [74]:
df_filtrado['popularity'] = df_filtrado['popularity'].astype(float)

In [75]:
# Exportar a formato parquet
df_filtrado.to_parquet('df_filtrado.parquet', index=False)

A partir de "df_filtrado" se creará un "df_functions" el cual tendrá solamente las columnas necesarias para responder a las funciones pedidas en el trabajo. Por otro lado, el archivo "df_filtrado.parquet" es aquel que se utilizará para seguir con el proceso de EDA.

In [76]:
# Creación del dataframe df_functions con las columnas requeridas para las funciones
df_functions = df_filtrado[['id', 'title', 'release_date', 'release_year', 'actors', 'director', 
                          'vote_average', 'vote_count', 'return', 'budget', 'revenue']].copy()

In [77]:
df_functions.head(5)

,id,title,release_date,release_year,actors,director,vote_average,vote_count,return,budget,revenue
0,862,Toy Story,1995-10-30,1995,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",John Lasseter,7.7,5415.0,12.451801,30000000,373554033.0
1,8844,Jumanji,1995-12-15,1995,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",Joe Johnston,6.9,2413.0,4.043035,65000000,262797249.0
2,15602,Grumpier Old Men,1995-12-22,1995,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...",Howard Deutch,6.5,92.0,0.000000,0,0.0
3,31357,Waiting to Exhale,1995-12-22,1995,"[Whitney Houston, Angela Bassett, Loretta Devi...",Forest Whitaker,6.1,34.0,5.090760,16000000,81452156.0
4,11862,Father of the Bride Part II,1995-02-10,1995,"[Steve Martin, Diane Keaton, Martin Short, Kim...",Charles Shyer,5.7,173.0,0.000000,0,76578911.0


In [78]:
df_functions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32484 entries, 0 to 45389
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            32484 non-null  int64         
 1   title         32484 non-null  object        
 2   release_date  32484 non-null  datetime64[ns]
 3   release_year  32484 non-null  int32         
 4   actors        32484 non-null  object        
 5   director      32484 non-null  object        
 6   vote_average  32484 non-null  float64       
 7   vote_count    32484 non-null  float64       
 8   return        32484 non-null  float64       
 9   budget        32484 non-null  int64         
 10  revenue       32484 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int32(1), int64(2), object(3)
memory usage: 2.9+ MB


In [79]:
df_expand.describe()

,budget,id,popularity,release_date,revenue,runtime,vote_average,vote_count,release_year,return
count,4.451000e+04,44510.000000,44510.000000,44510,4.451000e+04,44299.000000,44510.000000,44510.000000,44510.000000,4.451000e+04
mean,4.310829e+06,106777.475736,2.974195,1992-03-14 03:31:07.858908160,1.144374e+07,94.637080,5.653572,112.127679,1991.709054,6.728844e+02
min,0.000000e+00,2.000000,0.000000,1874-12-09 00:00:00,0.000000e+00,0.000000,0.000000,0.000000,1874.000000,0.000000e+00
25%,0.000000e+00,26028.500000,0.409902,1978-03-20 12:00:00,0.000000e+00,85.000000,5.000000,3.000000,1978.000000,0.000000e+00
50%,0.000000e+00,58781.500000,1.163834,2001-05-26 00:00:00,0.000000e+00,95.000000,6.000000,10.000000,2001.000000,0.000000e+00
75%,0.000000e+00,152602.500000,3.801981,2010-12-16 00:00:00,0.000000e+00,107.000000,6.800000,35.000000,2010.000000,0.000000e+00
max,3.800000e+08,469172.000000,547.488298,2020-12-16 00:00:00,2.787965e+09,1256.000000,10.000000,14075.000000,2020.000000,1.239638e+07
std,1.759694e+07,111574.496328,6.055897,NaN,6.499347e+07,37.408585,1.870198,496.278288,24.122523,7.541638e+04


Para reducir el tamaño de la muestra se empieza por eliminar aquellos registros que no presentan votos, o sea, "vote_count" igual a 0, y aquellos con "vote_average" igual a 0 tambien.

In [80]:
# Filtrar y eliminar las filas donde vote_average y vote_count son 0
df_functions = df_functions[(df_functions['vote_average'] != 0) & (df_functions['vote_count'] != 0)]

# Mostrar información actualizada del DataFrame
df_functions.info()


<class 'pandas.core.frame.DataFrame'>
Index: 30441 entries, 0 to 45387
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            30441 non-null  int64         
 1   title         30441 non-null  object        
 2   release_date  30441 non-null  datetime64[ns]
 3   release_year  30441 non-null  int32         
 4   actors        30441 non-null  object        
 5   director      30441 non-null  object        
 6   vote_average  30441 non-null  float64       
 7   vote_count    30441 non-null  float64       
 8   return        30441 non-null  float64       
 9   budget        30441 non-null  int64         
 10  revenue       30441 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int32(1), int64(2), object(3)
memory usage: 2.7+ MB


In [81]:
df_functions.head(10)

,id,title,release_date,release_year,actors,director,vote_average,vote_count,return,budget,revenue
0,862,Toy Story,1995-10-30,1995,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",John Lasseter,7.7,5415.0,12.451801,30000000,373554033.0
1,8844,Jumanji,1995-12-15,1995,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",Joe Johnston,6.9,2413.0,4.043035,65000000,262797249.0
2,15602,Grumpier Old Men,1995-12-22,1995,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...",Howard Deutch,6.5,92.0,0.000000,0,0.0
3,31357,Waiting to Exhale,1995-12-22,1995,"[Whitney Houston, Angela Bassett, Loretta Devi...",Forest Whitaker,6.1,34.0,5.090760,16000000,81452156.0
4,11862,Father of the Bride Part II,1995-02-10,1995,"[Steve Martin, Diane Keaton, Martin Short, Kim...",Charles Shyer,5.7,173.0,0.000000,0,76578911.0
5,949,Heat,1995-12-15,1995,"[Al Pacino, Robert De Niro, Val Kilmer, Jon Vo...",Michael Mann,7.7,1886.0,3.123947,60000000,187436818.0
6,11860,Sabrina,1995-12-15,1995,"[Harrison Ford, Julia Ormond, Greg Kinnear, An...",Sydney Pollack,6.2,141.0,0.000000,58000000,0.0
7,45325,Tom and Huck,1995-12-22,1995,"[Jonathan Taylor Thomas, Brad Renfro, Rachael ...",Peter Hewitt,5.4,45.0,0.000000,0,0.0
8,9091,Sudden Death,1995-12-22,1995,"[Jean-Claude Van Damme, Powers Boothe, Dorian ...",Peter Hyams,5.5,174.0,1.838576,35000000,64350171.0
9,710,GoldenEye,1995-11-16,1995,"[Pierce Brosnan, Sean Bean, Izabella Scorupco,...",Martin Campbell,6.6,1194.0,6.072311,58000000,352194034.0


In [83]:
# Exportar a formato parquet
df_functions.to_parquet('streaming_functions.parquet', index=False)